# Imports

In [1]:
!pip install datasets
from datasets import load_dataset, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import torch
import string
import re
import nltk
import pandas as pd
import json
nltk.download('stopwords')
from nltk.corpus import stopwords

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# 1. Baseline Model

In [ ]:
# Load training and validation datasets
dataset = load_dataset("Jinyan1/COLING_2025_MGT_en")

# Define preprocessing function
def preprocess_data(data):
    stop_words = set(stopwords.words('english'))
    text = ' '.join(data)
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Apply preprocessing to datasets
def preprocess_examples(example):
    example['text'] = preprocess_data([example['text']])
    return example

train_data = dataset['train'].map(preprocess_examples)
dev_data = dataset['dev'].map(preprocess_examples)

# Load RoBERTa tokenizer and define tokenization
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

train_data = train_data.map(tokenize_function, batched=True)
dev_data = dev_data.map(tokenize_function, batched=True)

# Set format for PyTorch
train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
dev_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Initialize RoBERTa model for classification
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none",
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=dev_data,
)

# Train the model
trainer.train()

# Evaluate on dev set
dev_results = trainer.evaluate()
print("Dev Results:", dev_results)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/588 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/287M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/286M [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

...

Map:   0%|          | 0/261758 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [20]:
### Load Test Dataset from JSONL File
# Replace 'path_to_your_test_file.jsonl' with the actual file path of your test dataset
jsonl_file_path = '/content/test_set_en.jsonl'

# Load JSONL file into a list of dictionaries
with open(jsonl_file_path, 'r') as f:
    test_data_list = [json.loads(line) for line in f]

# Print one example to verify loading
print("Example from Test Dataset:", test_data_list[0])

# Convert list of dictionaries to a Hugging Face Dataset
test_df = pd.DataFrame(test_data_list)
test_data = Dataset.from_pandas(test_df)

# Apply preprocessing to test dataset
test_data = test_data.map(preprocess_examples)

# Tokenize the test dataset
test_data = test_data.map(tokenize_function, batched=True)

# Set format for PyTorch
test_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Evaluate the model on the test dataset
test_results = trainer.evaluate(test_data)
print("Test Results:", test_results)

# Optional: Get predictions on the test dataset
predictions = trainer.predict(test_data)
predicted_labels = torch.argmax(predictions.predictions, axis=1)
print("Predicted Labels on Test Data:", predicted_labels)

Example from Test Dataset: {'testset_id': 0, 'text': 'Hello, Thanks for sharing your health concern with us. I have gone through your query and here are your answers: 1. If you have regular cycles, there is no further need to use any medication to regulate cycles. 2. Establishment of regular ovulation and timing of intercourse properly is necessary. 3. If you want to conceive quickly, you have to get further evaluation and plan management. Hope this helps.'}


Map:   0%|          | 0/73941 [00:00<?, ? examples/s]

KeyboardInterrupt: 